### Pytorch code

#### 코드에 새로 성별, cast 정보 등의 코드 추가

In [ ]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [ ]:
# 하이퍼파라미터 
class cfg: 
    gpu_idx = 0
    device = torch.device("cuda:{}".format(gpu_idx) if torch.cuda.is_available() else "cpu")
    top_k = 25
    seed = 42
    neg_ratio = 100
    test_size = 0.2

In [ ]:
# 시드 고정 
def seed_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    
seed_everything(cfg.seed)

In [ ]:
# 경로 설정
data_path = 'C:/Users/NTX550/Desktop/LG_AI공모전/final_dt/data/'
saved_path = 'C:/Users/NTX550/Desktop/LG_AI공모전/final_dt/code/saved/'
output_path = 'C:/Users/NTX550/Desktop/LG_AI공모전/final_dt/code/submission'

In [ ]:
# 데이터 전처리 (중복제거) 
# 참고 : drop_duplicates의 subset을 무엇으로 구성하냐에 따라서 제거되는 항목들이 다름 
# ex) 'profile_id', 'album_id' : 중복된 시청이력 모두 제거 / 'profile_id', 'album_id', 'log_time' : 같은 시간에 시청한 이력만 제거 
data = history_df[['profile_id', 'log_time', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id', 'log_time'])
data = data.sort_values(by = ['profile_id', 'log_time'])
data.reset_index(drop = True, inplace = True)

# 상영한 영상에 대해 Rating 1 부여 
data['rating'] = 1

cfg.n_users = data.profile_id.max()+1
cfg.n_items = data.album_id.max()+1

In [ ]:
# log_time 컬럼 중, 이상치 변환
def time_preprocessing(t):
    s = int(str(t)[-2:])
    m = int(str(t)[-4:-2])
    h = int(str(t)[-6:-4])
    if s >= 60:
        m += s//60
        s %= 60
    if m >= 60:
        h += m//60
        m %= 60
    if h >= 24:
        h = 23
        m = 59
        s = 59
    return str(t)[:-6] + str(h).zfill(2) + str(m).zfill(2) + str(s).zfill(2)

for df in [history_df, watch_df, search_df]:
    for col in ["ss_id", "log_time"]:
        df[col] = df[col].apply(lambda x: time_preprocessing(x))
        df[col] = df[col].apply(lambda x: datetime.datetime.strptime(str(x),"%Y%m%d%H%M%S"))

In [ ]:
# history_df와 search_df를 merge... 단, log_time이 다름에 주의해야함.
temp_h = history_df.copy()
temp_s = search_df.copy()

# join을 위해 년월일 시까지만 고려.
temp_h["log_for_join"] = temp_h["log_time"].apply(lambda x: datetime.datetime.strftime(x, "%Y%m%d%H"))
temp_s["log_for_join"] = temp_s["log_time"].apply(lambda x: datetime.datetime.strftime(x, "%Y%m%d%H"))
temp_s.drop(columns=["log_time"], inplace=True)

merged_df = pd.merge(temp_h, temp_s, how='left', 
                     on=["profile_id","ss_id", "log_for_join", "album_id"])
merged_df

In [ ]:
# merged_df와 watch_df를 merge ... 단, log_time이 다름.
temp_m = merged_df.copy()
temp_w = watch_df.copy()

temp_m = pd.merge(temp_m, temp_w, how='inner',
                  on=["profile_id","ss_id", "album_id"])
temp_m

In [ ]:
# log_diff 와 watch_time 간, 차이가 큰 경우를 중복데이터 및 이상치로 판단하여 해당 rows 제거.
# 일단 log_diff가 양수인 경우만 고려.
temp_m = temp_m[temp_m["log_diff"] >= 0]
temp_m

In [ ]:
# 콘텐츠 전체 길이에 대한 시청 비율 컬럼을 추가
temp_m["time_ratio"] = temp_m["watch_time"] / temp_m["total_time"]

In [ ]:
# 데이터 전처리 (중복제거) 
# 참고 : drop_duplicates의 subset을 무엇으로 구성하냐에 따라서 제거되는 항목들이 다름 
# ex) 'profile_id', 'album_id' : 중복된 시청이력 모두 제거 / 'profile_id', 'album_id', 'log_time' : 같은 시간에 시청한 이력만 제거 
data = history_df[['profile_id', 'log_time', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
data['rating'] = 1

# Matrix 형태로 변환 
train = train.to_numpy()
matrix = sparse.lil_matrix((cfg.n_users, cfg.n_items))
for (p, _, i, r) in tqdm(train):
    matrix[p, i] = r

In [ ]:
#기존과 다르게 cast에 대한 정보 추가

le = LabelEncoder()
meta_df['cast_1'] = le.fit_transform(meta_df['cast_1'])
item_features['cast_1'] = meta_df[['cast_1']].to_dict()['cast_1']

cfg.n_cast = meta_df['cast_1'].nunique()
cfg.n_continuous_feats = 1

In [ ]:
# 새로 작성한 코드만 

cast_embedding_mlp = self.cast_embeddig(feats[2])
        ###########################################################################
input_feature = torch.cat((user_embedding_mlp, item_embedding_mlp, genre_embedding_mlp, cast_embedding_mlp, feats[0].unsqueeze(1)), -1)
mlp_output = self.mlp_layers(input_feature)
        
output = torch.cat([mlp_output, mf_output], dim=-1)
output = self.affine_output(output).squeeze(-1)

features = []
for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
        features.append(item_features['cast_1'][item_id])
        
UIdataset[user_id].append(np.array(features))

In [ ]:
# 학습 및 검증 데이터 분리
train, valid = train_test_split(
    data, test_size=cfg.test_size, random_state=cfg.seed)

학습 데이터 크기: (719401, 4)
검증 데이터 크기: (179851, 4)


In [ ]:
# Matrix 형태로 변환(가로 축 : 유저 데이터, 세로 축 : 아이템 데이터) - 추천 시스템의 형태
train = train.to_numpy()
matrix = sparse.lil_matrix((cfg.n_users, cfg.n_items))
for (p, _, i, r) in train:
    matrix[p, i] = r
    
train = sparse.csr_matrix(matrix)
train = train.toarray()
print("train 형태: \n", train)

  0%|          | 0/719401 [00:00<?, ?it/s]

train 형태: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# 유저 특징 정보 추출 
profile_df = profile_df.set_index('profile_id')
user_features = profile_df[['age']].to_dict()

# 아이템 특징 정보 추출 
meta_df = meta_df.set_index('album_id')

# 범주형 데이터를 수치형 데이터로 변경 
le = LabelEncoder()
meta_df['genre_mid'] = le.fit_transform(meta_df['genre_mid'])
item_features = meta_df[['genre_mid']].to_dict()

# 추출한 특징 정보의 속성을 저장 
cfg.n_genres = meta_df['genre_mid'].nunique()
cfg.n_continuous_feats = 1 

# 성별 데이터 추가 
profile_df['sex'] = le.fit_transform(profile_df['sex'])
item_features1 = profile_df[['sex']].to_dict()

cfg.n_sex = profile_df['sex'].nunique()

In [ ]:
def build_graph(self):

    self.user_embedding_mf = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim)
    self.item_embedding_mf = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim)
        
    self.user_embedding_mlp = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim)
    self.item_embedding_mlp = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim)
                
    self.genre_embeddig = nn.Embedding(num_embeddings=self.n_genres, embedding_dim=self.n_genres//2)
    self.sex_embedding = nn.Embedding(num_embeddings=self.n_sex, embedding_dim=self.n_sex)
        
    self.mlp_layers = nn.Sequential(
        nn.Linear(2*self.emb_dim + self.n_genres//2 + self.n_sex + self.n_continuous_feats, self.layer_dim), 
        nn.ReLU(), 
        nn.Dropout(p=self.dropout), 
        nn.Linear(self.layer_dim, self.layer_dim//2), 
        nn.ReLU(), 
        nn.Dropout(p=self.dropout))

    self.affine_output = nn.Linear(self.layer_dim//2 + self.emb_dim, 1)
    self.apply(self._init_weights)
        

def _init_weights(self, module): # pytorch 가중치 초기화
    if isinstance(module, nn.Embedding):
        normal_(module.weight.data, mean=0.0, std=0.01)
    elif isinstance(module, nn.Linear):
        normal_(module.weight.data, 0, 0.01)
        if module.bias is not None:
            module.bias.data.fill_(0.0)
    
def forward(self, user_indices, item_indices, feats):

    user_embedding_mf = self.user_embedding_mf(user_indices)
    item_embedding_mf = self.item_embedding_mf(item_indices)
    mf_output = torch.mul(user_embedding_mf, item_embedding_mf)
        
    user_embedding_mlp = self.user_embedding_mlp(user_indices)
    item_embedding_mlp = self.item_embedding_mlp(item_indices)
    genre_embedding_mlp = self.genre_embeddig(feats[1])
    sex_embedding_mlp = self.sex_embedding(feats[2])
    input_feature = torch.cat((user_embedding_mlp, item_embedding_mlp, genre_embedding_mlp, sex_embedding_mlp, feats[0].unsqueeze(1)), -1)
    mlp_output = self.mlp_layers(input_feature)
        
    output = torch.cat([mlp_output, mf_output], dim=-1)
    output = self.affine_output(output).squeeze(-1)
    return output

In [2]:
# 코드 저작권으로 개인 작성 코드 위주로 첨부